In [ ]:
!apt-get install --no-install-recommends -y openmpi-bin libopenmpi-dev libopencv-dev python-opencv && ln -sf /usr/lib/x86_64-linux-gnu/libmpi_cxx.so /usr/lib/x86_64-linux-gnu/libmpi_cxx.so.1 && ln -sf /usr/lib/x86_64-linux-gnu/openmpi/lib/libmpi.so /usr/lib/x86_64-linux-gnu/openmpi/lib/libmpi.so.12 && ln -sf /usr/lib/x86_64-linux-gnu/libmpi.so /usr/lib/x86_64-linux-gnu/libmpi.so.12 && pip install cntk

In [2]:
import cntk as C

model_path = '/kaggle/input/pretrained-deepdanboru/model.cntk'

ctnk_model = C.load_model(model_path)
ctnk_model.save('./model.onnx', format=C.ModelFormat.ONNX)

/opt/conda/lib/python3.6/site-packages/cntk/cntk_py_init.py:56: UserWarning: Unsupported Linux distribution (sion_id="9"-tps://www.debian.org/). CNTK supports Ubuntu 16.04 and above, only.
  warnings.warn('Unsupported Linux distribution (%s-%s). CNTK supports Ubuntu 16.04 and above, only.' % (__my_distro__, __my_distro_ver__))
/opt/conda/lib/python3.6/site-packages/cntk/cntk_py_init.py:98: UserWarning: 

################################################ Missing optional dependency (GPU-Specific) ################################################
   CNTK may crash if the component that depends on those dependencies is loaded.
   Visit https://docs.microsoft.com/en-us/cognitive-toolkit/Setup-Linux-Python#optional-gpu-specific-packages for more information.
############################################################################################################################################
If you intend to use CNTK without GPU support, you can ignore the (likely) GPU-specific warning!


In [3]:
!pip install -q onnxruntime

     |████████████████████████████████| 3.6MB 2.8MB/s eta 0:00:01


In [4]:
import onnxruntime
import numpy as np

In [5]:
ort_session = onnxruntime.InferenceSession("./model.onnx")

x = np.random.rand(1, 3, 299, 299).astype(np.float32)
# compute ONNX Runtime output prediction
ort_inputs = {ort_session.get_inputs()[0].name: x}
ort_outs = ort_session.run(None, ort_inputs)

# compute the ctnk output
cntk_out = ctnk_model.eval(x)

In [6]:
np.testing.assert_allclose(np.array(cntk_out), ort_outs[0], rtol=1e-03, atol=1e-05)